In [ ]:
%pip install --quiet langchain langgraph

In [ ]:
from pydantic import BaseModel
from typing import List

class QueryResult(BaseModel):
    title: str = None
    url: str = None
    resume: str = None

class ReportState(BaseModel):
    user_input: str = None
    final_response: str = None
    queries: List[str] = []

In [ ]:
import os
import dotenv

from langchain_openai import ChatOpenAI
from langgraph.graph import START, END, StateGraph
from langgraph.types import Send

dotenv.load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(model="gpt-4.1")

reasoning = {
    "effort": "medium",  # 'low', 'medium', or 'high'
    "summary": "auto",  # 'detailed', 'auto', or None
}

reasoning_llm = ChatOpenAI(
    model="o4-mini", reasoning=reasoning
)

In [ ]:
# Nodes
def build_first_queries(state: ReportState):
    

In [ ]:


builder = StateGraph(ReportState)
graph = builder.compile()

In [ ]:
user_input = "Write a report on the impact of climate change on polar bears."

graph.invoke({"user_input": user_input})